<a href="https://colab.research.google.com/github/yu072333/114-/blob/main/%E7%A8%8B%E5%BC%8F%E8%AA%9E%E8%A8%80%E4%BD%9C%E6%A5%AD4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
!pip install --upgrade google-generativeai

In [23]:
# ===========================================
# 💰 PTT 省錢版全文分析 x Google Sheet x Gemini x Gradio
# ===========================================
# ---- 基本匯入 ----
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
import jieba
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import Counter, defaultdict
import gradio as gr
import matplotlib.pyplot as plt
import os
import tempfile
import google.generativeai as genai
import concurrent.futures

In [24]:
# ---- Google Sheet 設定 (請根據您的 Sheet 網址修改) ----
creds, _ = default()
gc = gspread.authorize(creds)
# ⚠️ 請將此 URL 替換為您實際的 Google Sheet 網址
SHEET_URL ="https://docs.google.com/spreadsheets/d/16kQ4F7zvCw0vfo7a3JUWeTeubgbGkWpc0GsMecl_VRQ/edit?gid=0#gid=0"
sh = gc.open_by_url(SHEET_URL)
worksheet = sh.worksheet("PTT_ARTICLE")


In [25]:
# ---- Gemini API 設定 ----
from google.colab import userdata
import google.generativeai as genai

api_key = userdata.get("gemini")
genai.configure(api_key=api_key)

In [26]:
 #---- PTT 省錢版爬蟲函式 ----
def crawl_ptt_lifeismoney(pages=3):
    """爬取 PTT 省錢版前幾頁文章及全文"""
    base_url = "https://www.ptt.cc/bbs/Lifeismoney/index.html"
    headers = {'User-Agent': 'Mozilla/5.0'}
    articles = []

    for _ in range(pages):
        res = requests.get(base_url, cookies={"over18": "1"}, headers=headers, timeout=5)
        soup = BeautifulSoup(res.text, "html.parser")

        for entry in soup.select(".r-ent"):
            title_tag = entry.select_one(".title a")
            author = entry.select_one(".author").text.strip()
            date = entry.select_one(".date").text.strip()

            if title_tag:
                title = title_tag.text.strip()
                href = "https://www.ptt.cc" + title_tag["href"]

                try:
                    art_res = requests.get(href, cookies={"over18": "1"}, headers=headers, timeout=5)
                    art_soup = BeautifulSoup(art_res.text, "html.parser")
                    content = art_soup.select_one("#main-content").text.strip()
                    content = re.sub(r"--\s.*", "", content, flags=re.S)
                    content = content[:300]  # 只取前 300 字避免太慢
                except:
                    content = "(無法取得內文)"
            else:
                title = "(已刪除)"
                href = ""
                content = "(無法取得內文)"

            articles.append({
                "title": title,
                "author": author,
                "date": date,
                "href": href,
                "content": content
            })

        prev = soup.find("a", string="上頁")
        if not prev: break
        base_url = "https://www.ptt.cc" + prev["href"]

    return pd.DataFrame(articles)

In [27]:
# ---- 詞頻與 TF-IDF 分析 ----
def analyze_text(df):
    stopwords = set([
        '的','了','是','在','我','你','他','她','之','一個','和','討論','分享', '優惠','活動','折價','折扣','情報','好康','買','賣','元','錢','省錢', '回饋','期限','連結','點數','商品','使用','沒有','這個','可以','大家', '請益','問題','發問','謝謝','求助'])
    word_counts = Counter()
    documents = []

    for text in df["content"]:
        cleaned = re.sub(r"[^\w\s]", "", text)
        words = [w for w in jieba.lcut(cleaned) if len(w.strip())>1 and w not in stopwords]
        word_counts.update(words)
        documents.append(" ".join(words))

    top_freq = word_counts.most_common(20)

    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(documents)
    features = vectorizer.get_feature_names_out()
    avg_scores = defaultdict(float)

    for row in tfidf_matrix.toarray():
        for i, val in enumerate(row):
            avg_scores[features[i]] += val
    for k in avg_scores: avg_scores[k] /= len(documents)

    top_tfidf = sorted(avg_scores.items(), key=lambda x:x[1], reverse=True)[:20]

    return top_freq, top_tfidf

# ---- Gemini 洞察函式（加入 timeout） ----
def generate_insight(text_summary):
    """用 Gemini 生成 5 句洞察 + 120 字結論"""
    prompt = f"""
    根據以下文字內容，請生成五句精煉洞察，並以約120字給出總結性評論。
    ---
    {text_summary}
    """
    model = genai.GenerativeModel("gemini-1.5-flash")

    def call_model():
        return model.generate_content(prompt).text

    with concurrent.futures.ThreadPoolExecutor() as executor:
        future = executor.submit(call_model)
        try:
            return future.result(timeout=15)  # 最多等 15 秒
        except concurrent.futures.TimeoutError:
            return "⚠️ Gemini 回應逾時（超過15秒）"

In [28]:

# ---- 主函式 ----
def main(pages=3, topn=10):
    df = crawl_ptt_lifeismoney(pages)
    worksheet.clear()
    worksheet.update([df.columns.values.tolist()] + df.values.tolist())

    top_freq, top_tfidf = analyze_text(df)

    # 回寫統計表
    try:
        stat_ws = sh.worksheet("統計結果")
    except:
        stat_ws = sh.add_worksheet(title="統計結果", rows=100, cols=6)

    stat_data = [["排名","詞","詞頻","TF-IDF權重"]]
    for i in range(min(topn, len(top_freq))):
        word_f, count = top_freq[i]
        word_t, score = top_tfidf[i]
        stat_data.append([i+1, word_f, count, f"{score:.4f}"])
    stat_ws.clear()
    stat_ws.update(stat_data)

    insight_input = " ".join(df["content"].head(5))
    summary = generate_insight(insight_input)

    return df.head(10), summary, stat_data[1:]


In [29]:
# ---- Gradio 介面 ----
def run_ui(pages, topn):
    df, summary, stats = main(int(pages), int(topn))

    fig, ax = plt.subplots(figsize=(6,3))
    words = [row[1] for row in stats]
    counts = [int(row[2]) for row in stats]
    ax.barh(words[::-1], counts[::-1])
    plt.title("Top 詞頻統計")
    plt.tight_layout()
    tmp = tempfile.NamedTemporaryFile(delete=False, suffix=".png")
    plt.savefig(tmp.name)

    return (
        df.to_markdown(),
        summary,
        tmp.name
    )

demo = gr.Interface(
    fn=run_ui,
    inputs=[
        gr.Number(value=3, label="爬取頁數"),
        gr.Number(value=10, label="顯示前 N 熱詞")
    ],
    outputs=[
        gr.Textbox(label="📄 PTT 省錢版前10篇文章資料 (前10筆)", lines=15),
        gr.Textbox(label="💡 Gemini 洞察摘要 + 結論", lines=8),
        gr.Image(label="📊 詞頻圖表")
    ],
    title="PTT 省錢版 全文分析 x Google Sheet x Gemini",
    description="一鍵爬取、統計與生成 AI 洞察報告"
)

demo.launch()

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://beb83b1e9f17acfd4a.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
